In [ ]:
source(here::here("R/setup.R"))
library(lubridate)

# 日付・時間データの取り扱い

ビールへの支出データ、土砂災害・雪崩メッシュデータを利用します。

- 経過時間
- タイムゾーン

期間内のある瞬間の情報

特定のイベントからの経過時間

ビールの売り上げに対して、何が効いているか、効いていないか予測を立ててみましょう

* 平日・休日
    * より厳密には週末かそうでないか… そんなに効かない。土日は効果あり
    * 連休の数
        * 平日金曜日は0.5?
* 夏休み？？
* 関係しそうではないもの
    * 月初、月末
    
## 要素の分解

日付や時間のデータは複数の要素で構成されます。例えば「2019年7月10日」であれば、年月日をそれぞれ分けて2019、7、10に分解可能です。これに時間が加わればさらに時分秒の要素に分解することもできます。

ビールの売り上げを考えたとき、大事な日付の要素は何でしょうか。

ということがない限り、年の影響は小さそうです。小さなデータセットでは一年分しかない場合もあります（分散0）。

日付もそれほど重要ではなさそうです。むしろ日付によって変わる平日・休日の違いが影響しそうですが、これについてはあとで処理を加えていくことにします。

ビールの支出データに記録されている日付の曜日をダミーコーディングする例を考えてみましょう。曜日は7つの値を取りますが、コントラスト関数は6つのダミー変数で曜日を表現することになります。

6列... 該当する曜日で1, そうでない場合に0

まずは年月日からなる日付の変数から曜日を取り出す必要があります。

In [ ]:
mod_fml <- formula(expense ~ date + weatherdaytime_06_00_18_00)

日付を記録するdate列の要素が分解され、新たな特徴量として追加されました。それでは曜日のダミーコーディングを実行します。

In [ ]:
df_baked_split_date %>% 
  recipe(expense ~ .) %>% 
  step_dummy(date_dow) %>% 
  prep(training = df_baked_split_date) %>% 
  bake(new_data = df_baked_split_date) %>% 
  select(starts_with("date_dow"), everything())

In [ ]:
rep_split_date <- 
  df_beer2018q2 %>% 
  recipe(expense ~ .) %>% 
  step_date(date)

df_beer_prep <- 
  rep_split_date %>% 
  prep(training = df_beer2018q2) %>% 
  juice(expense, starts_with("date"), temperature_average)

df_beer_prep

In [ ]:
df_baked_split_date <- 
  df_beer2018q2 %>% 
  recipe(mod_fml) %>% 
  step_date(date) %>% 
  prep(training = df_beer2018q2) %>% 
  bake(new_data = df_beer2018q2)

glimpse(df_baked_split_date)

今度は分解した曜日の情報をもとに、ビールの売り上げは「翌日に仕事が控えている曜日よりも休日の方が増えそうだ」という直感を調べてみましょう。

In [ ]:
df_baked_split_date <- 
  df_baked_split_date %>% 
  mutate(is_weekend = if_else(date_dow %in% c("土", "日"),
                              TRUE,
                              FALSE))

In [ ]:
df_baked_split_date %>% 
  ggplot(aes(date_dow, expense)) +
  geom_boxplot(aes(color = is_weekend), outlier.shape = NA) +
  geom_jitter(aes(color = is_weekend), alpha = 0.3) +
  scale_color_ds() +
  facet_wrap(~ date_month)

目論見通り、どの月でも平日よりも休日の方が支出が増えているようです。また、8月は週ごとに変動が大きく、9月では平日との差がほとんどないということもグラフから読み取れます。新たな特徴量の作成と関係の図示により、経験的な推論を確認するだけでなく、モデルに対する新たな洞察も得ることができました。

In [ ]:
# 日本の祝日判定に次の項目が利用可能
stringr::str_subset(timeDate::listHolidays(), "^JP")

## カレンダー（祝日）

人の行動による影響を受けるデータでは、日付の平日・休日を区別することが重要になることが多いです。それは平日と休日では人々の行動様式が異なると考えられるためです。

一方でハザードデータのように自然活動を扱うデータではこの要素が影響するとは考えられません。

データが記録された日付が平日か休日

7月から9月にかけては

海の日（7月第3月曜日）
山の日 (8月11日) ... 2016年に設立。
敬老の日 (9月第3月曜日)
秋分の日 (秋分日... )

が祝日です。

また、この期間で考慮すべき日付として「お盆」の期間があります。多くの企業でこの期間は夏休みとなっていると考えられます。8月13日から15日のデータも他の日付と区別できるようにしておきます。

In [ ]:
df_beer_prep <- 
  df_beer_prep %>% 
  recipe(expense ~ .) %>% 
  step_holiday(date,
               holidays = timeDate::listHolidays("JP") %>% 
                 str_subset("UmiNoHi|KeirouNOhi|ShuubunNoHi")) %>% 
  prep() %>% 
  bake(new_data = df_beer_prep) %>%
  # timeDate::listHolidays() が山の日に未対応なのでフラグを作る処理を用意する
  mutate(date_JPYamaNoHi = as.numeric(date == ymd("2018-08-11")),
         is_obon = between(date, ymd("2018-08-13"), ymd("2018-08-15")))

df_beer_prep

平日であれば1を与える列を追加します。

In [ ]:
df_beer_prep <- 
  df_beer_prep %>% 
  mutate(is_weekday = if_else(date_dow %in% c("土", "日"),
                              0,
                              if_else(date_JPKeirouNOhi == 1 | date_JPShuubunNoHi == 1 | date_JPUmiNoHi == 1 | date_JPYamaNoHi == 1,
                                      0,
                                      if_else(is_obon == 1,
                                              0,
                                              1))
                              )) %>% 
  select(expense, is_weekday, date_month, temperature_average)

df_beer_prep

In [ ]:
df_beer_baked <- 
  df_beer_prep %>% 
  recipe(expense ~ .) %>% 
  step_log(expense, temperature_average, base = 10) %>% 
  prep() %>% 
  juice()

## 自己相関
## 季節成分・周期成分

weeks

df_beerの方で??
df_hazard

<!--## タイムゾーン -->